# Расчет УЭЦН

Расчет системы УЭЦН можно разделить на 2 части: определение работы непосредственно ЭЦН и остального оборудования (сепаратора, протектора, ПЭД, кабеля, ТМПН, СУ)

## Расчет ЭЦН

Расчет ведется интегрированием по ступеням с применением поправок на вязкость, плотность смеси, при необходимости калибровочных коэффициентов.

Исходными данными являются параметры насоса, давление и температура на приеме, модифицированные PVT свойства газожидкостной смеси после сепарации.

Алгоритм для i-ой ступени:

1. Расчет PVT свойств смеси по корреляционным зависимостям модели BlackOil
2. Вычисление коэффициента, учитывающего вредное влияние свободного газа только для 1-ой ступени
$$C_{gas} = f(gas fraction d, gas degr type)$$  
3. Определение дебита газожидкостной смеси в условиях рассматриваемой ступени
$$Q_{mix_{i}} = Q_{mix_{i}}(P_i,T_i) $$
4. Определение дебита газожидкостной смеси с учетом деградации напорной характеристики УЭЦН с помощью калибровочного коэффициента на дебиту $C_{Q}$
$$Q_{mix_{i}}^{degr} = Q_{mix_{i}} * C_{Q}$$
5. Определение напора, развиваемого ЭЦН, по кривым с помощью базы насосов и поправки на вязкость Института гидравлики $H_{PI}$, коэффициента калибровки по напору $C_{H}$, коэффициента, учитывающего вредное влияние свободного газа $C_{gas}$.
$$H_{mix_{i}}^{degr} = H_{PI_{i}} * C_{H} * C_{gas}$$
$$i > 1 \to C_{gas} = 1 $$
6. Вычисление общего напора ЭЦН (суммирование напоров всех пройденных ступеней)
$$H_{mix}^{degr} = \sum_{1}^{i} H_{mix_{i}}^{degr} = \sum_{1}^{i-1} H_{mix_{i}}^{degr}  + H_{mix_{i}}^{degr}$$
7. Расчет перепада давления, развиваемого данной ступенью с учетом плотности ГЖС
$$dP_{mix_{i}} = \rho_{mix_{i}} * H_{mix_{i}}^{degr} * g$$
8. Оценим работу, совершаемую насосом по перекачке жидкости, т.е. вычислим полезную (гидравлическую мощность), переданную ГЖС.
$$N_{fluid_{i}} = Q_{mix_{i}}^{degr} * dP_{mix_{i}}$$
9. Вычислим суммарную полезную мощность, отданную ГЖС ступенями до i-ой включительно. 
$$N_{fluid} = \sum_{1}^{i} N_{fluid_{i}} = \sum_{1}^{i-1} N_{fluid_{i}} +  N_{fluid_{i}}$$
10. Оценим мощность, потребляемую насосом с вала, воспользуясь кривой из базы ЭЦН и поправкой на вязкость Института Гидравлики, относительной плотностью ГЖС и калибровочным коэффициентом на мощность $C_{N}$
$$N_{ESP_{i}} = N_{PI_{i}} * \gamma_{mix_{i}} * C_{N} $$
где 
$$\gamma_{mix_{i}} = \frac{\rho_{mix_{i}}}{\rho_{wat_{н.у}}}$$
10. Оценим потребляемую мощность всех ступеней до i-ой включительно
$$N_{ESP} = \sum_{1}^{i} N_{ESP{i}} = \sum_{1}^{i-1} N_{ESP{i}} +  N_{ESP{i}}$$
11. Оценим КПД ступени в данных условиях
$$\eta_i = \frac{N_{fluid_{i}}}{N_{ESP_{i}}} $$
12. Оценим общий КПД ЭЦН
$$\eta = \frac{N_{fluid}}{N_{ESP}} $$
13. Рассчитаем перепад температур на i-ой ступени
$$dT_i = \frac{H_{mix_{i}}^{degr} * g * (1 - \eta_i)}{c_{p_{mix_{i}}} * \eta_i}$$
14. Общий нагрев смеси - перепад температур
$$dT = \sum_{1}^{i} dT_{{i}} = \sum_{1}^{i-1} dT_{{i}} +  dT_{{i}}$$
15. Термобарические условия на входе следующей ступени или выходе из насоса
$$P_{i+1} = P_{i} + dP_{mix_{i}}$$
$$T_{i+1} = T_{i} + dT_{i}$$

На этом алгоритм, который повторяется на каждой ступения, заканчивается. Для расчета энергетической составляющей потребляемая мощность ЭЦН (забираемая с вала ЭЦН - $N_{ESP}$) используется в дальнейшем.

## Расчет системы ЭЦН

Мощность, потребляемая системой ЭЦН считается последовательно по цепочке элементов, входящих в данных комплекс оборудования: насос, сепаратор, протектор, ПЭД, кабель, ТМПН, СУ.

1. Мощность потребляемая с вала, рассчитывается с учетом ЭЦН, протектора и газосепаратора. Причем два последних элемента забирают фиксированное значения Вт, 400 и 500 соответственно (в будущем необходимо запрограммировать их нагрузочную характеристику)
$$N_{shaft} = N_{ESP} + N_{protector} + N_{gassep}$$
2. Затем исходя из постоянного значения КПД ПЭД, вычисляется электричекая  мощность, приходящая в электродвигатель (характеристика асинхронного ПЭД есть в UniflocVBA, но не проверена и потому не встроена в расчет)
$$N_{motor} = \frac{N_{shaft}}{\eta_{motor}}$$
3. Для того, чтобы оценить потери в питающем кабеле, оценим ток двигателя
$$I_{motor} = \frac{N_{motor}}{1.73 * U * cos\phi} $$
*где $U$  - напряжение на двигателе*
4. По данным тока и сопротивления кабеля найдем падения напряжение в кабеле
$$dU_{cable}  = I_{motor} * \rho_{cable} * H_{MD PUMP}$$
где 
$H_{PUMP}$ - измеренная глубина спуска насоса вдоль ствола скважины, которая равна длине кабеля 
$$H_{PUMP} = L_{cable}$$
5. Напряжение на трасформаторе
$$U_{transform} = U_{motor} + dU_{cable}$$
6. По падению напряжения оценим мощность рассеиваемую кабелем в тепло, учитываем 3 жилы.
$$dN_{cable} = 3 * dU_{cable} * I_{motor} $$
7. Найдем электрическую  мощность на входе в кабель
$$N_{cable} = N_{motor} + dN_{cable}$$
8. Оценим потери мощности в трансформаторе
$$dN_{transform} = (1 - \eta_{CS}) * N_{cable}$$
9. И в станции управления (Control Station - CS)
$$dN_{CS} = (1 - \eta_{transform}) * (N_{cable} + dN_{transform}) $$
10. Мощность, приходящая на СУ, т.е. активная мощность, потребляемая всей системой УЭЦН.
$$N_{CS} =  N_{cable} + dN_{transform} + dN_{CS}$$
11. Рассчитаем номинальную мощность двигателя на данной частоте
$$N_{motor_{nom_F}} = N_{motor_{nom}} * \frac{F}{F_{nom}}$$
12. После можем найти загрузку электродвигателя
$$Load_{motor} = \frac{N_{motor}}{N_{motor_{nom_F}}}$$
13. Дополнительно КПД всей системы ЭЦН может быть вычислен по формуле
$$\eta_{overall} = \frac{N_{fluid}}{N_{CS}}$$

На этом расчет системы ЭЦН закончен. Для адаптации/восстановления/прогноза может быть использована следующая функция ошибки

$$LOSS = (P_{buf}^{fact} - P_{buf}^{calc})^2+(N_{CS}^{fact} - N_{CS}^{calc})^2$$